In [1]:
import json
import os
from os import path
import time
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import spacy

import mysql.connector

sql_config = {
    'user': 'root',
    'password': 'see-cret',
    'host': 'localhost',
    'database': 'tcc'
}

DIRETORIO_PDF = 'BauruPDF/'
DIRETORIO_TXT = 'BauruTXT/'
DIRETORIO_DOC = 'BauruDOC/'
DIRETORIO_PAR = 'BauruPAR/'
DIRETORIO_JSON = 'BauruJSON/'

nlp_exclude_list = ['parser',
                    'ner',
                    'entity_linker',
                    'entity_ruler',
                    'textcat',
                    'textcat_multilabel',
                    'lemmatizer',
                    'trainable_lemmatizer',
                    'morphologizer',
                    'attribute_ruler',
                    'senter',
                    'sentencizer',
                    'tok2vec',
                    'transformer']
nlp = spacy.load('pt_core_news_lg', exclude = nlp_exclude_list)

In [2]:
time_in = time.time()
licitacoes_json = []
for l in os.listdir(DIRETORIO_JSON):
    with open(path.join(DIRETORIO_JSON, l), 'r') as arquivo:
        licitacoes_json.append(json.load(arquivo))
time_out = time.time()
print('tempo de carregamento dos .json', time_out - time_in)

tempo de carregamento dos .json 2.285004138946533


In [3]:
with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    try:
        cursor.execute('SELECT * FROM licitacao')
        for d in cursor:
            print(d)
        cursor.close()
    except mysql.connector.Error as e:
        print('ERRO', e)

In [4]:
time_in = time.time()

for licitacao in tqdm(licitacoes):
    # Verifica existencia de modalidade
    with mysql.connector.connect(**sql_config) as connection:
        cursor = connection.cursor()
        
        try:
            titulo_modalidade = licitacao['titulo_modalidade']
            cursor.execute(f'SELECT modalidade_id FROM modalidade WHERE descricao LIKE "{titulo_modalidade}"')
            cursor.fetchall()
            if cursor.rowcount == 0:
                cursor.execute(f'INSERT INTO modalidade(descricao) VALUES ("{titulo_modalidade}")')
                cursor.fetchall()
            connection.commit()
        except mysql.connector.Error as e:
            print('ERRO', e)
            connection.rollback()
            break
        finally:
            cursor.close()
        
time_out = time.time()
print(time_out - time_in)

100%|██████████| 3555/3555 [00:41<00:00, 86.62it/s] 

41.10868978500366


In [5]:
time_in = time.time()
with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    for licitacao in licitacoes:
        try:
            interessado = licitacao['interessado']
            cursor.execute(f'SELECT interessado_id FROM interessado WHERE descricao LIKE "{interessado}"')
            cursor.fetchall()
            if cursor.rowcount == 0:
                cursor.execute(f'INSERT INTO interessado(descricao) VALUES ("{interessado}")')
                cursor.fetchall()
            connection.commit()
        except mysql.connector.Error as e:
            print('ERRO', e)
            connection.rollback()
            break
    cursor.close()
time_out = time.time()
print(time_out - time_in)

4.435302972793579


In [6]:
time_in = time.time()
with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    for licitacao in tqdm(licitacoes):
        try:
            tipo = licitacao['tipo']
            cursor.execute(f'SELECT tipo_id FROM tipo WHERE descricao LIKE "{tipo}"')
            cursor.fetchall()
            if cursor.rowcount == 0:
                cursor.execute(f'INSERT INTO tipo(descricao) VALUES ("{tipo}")')
                cursor.fetchall()
            connection.commit()
        except mysql.connector.Error as e:
            print('ERRO', e)
            connection.rollback()
            break
    cursor.close()
time_out = time.time()
print(time_out - time_in)

100%|██████████| 3555/3555 [00:02<00:00, 1525.72it/s]

2.3483805656433105


In [7]:
time_in = time.time()
with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    for publicacao in tqdm(publicacoes):
        try:
            titulo = publicacao['titulo']
            cursor.execute(f'SELECT publicacao_titulo_id FROM publicacao_titulo WHERE descricao LIKE "{titulo}"')
            cursor.fetchall()
            if cursor.rowcount == 0:
                cursor.execute(f'INSERT INTO publicacao_titulo(descricao) VALUES ("{titulo}")')
                cursor.fetchall()
            connection.commit()
        except mysql.connector.Error as e:
            print('ERRO', e)
            connection.rollback()
            break
    cursor.close()
time_out = time.time()
print(time_out - time_in)

100%|██████████| 6593/6593 [00:05<00:00, 1156.86it/s]

5.716196060180664


In [8]:
time_in = time.time()
with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    for licitacao in tqdm(licitacoes):
        try:
            titulo_modalidade = licitacao['titulo_modalidade']
            cursor.execute(f'SELECT modalidade_id FROM modalidade WHERE descricao LIKE "{titulo_modalidade}"')
            modalidade_id = cursor.fetchone()[0]
            cursor.fetchall()
            
            interessado = licitacao['interessado']
            cursor.execute(f'SELECT interessado_id FROM interessado WHERE descricao LIKE "{interessado}"')
            interessado_id = cursor.fetchone()[0]
            
            tipo = licitacao['tipo']
            cursor.execute(f'SELECT tipo_id FROM tipo WHERE descricao LIKE "{tipo}"')
            tipo_id = cursor.fetchone()[0]
            cursor.fetchall()
            
            identificador = licitacao['identificador']
            numero_modalidade = licitacao['titulo_numero']
            ano_modalidade = licitacao['titulo_ano']
            numero_processo = licitacao['processo_numero']
            ano_processo = licitacao['processo_ano']
            
            cursor.execute(f'INSERT INTO licitacao(identificador, modalidade_id, numero_modalidade, ano_modalidade, tipo_id, interessado_id, numero_processo, ano_processo)' +
                          f'VALUES ({identificador}, {modalidade_id}, {numero_modalidade}, {ano_modalidade}, {tipo_id}, {interessado_id}, {numero_processo}, {ano_processo})')
            
        except mysql.connector.Error as e:
            print('ERRO', e)
            connection.rollback()
            break
    cursor.close()
    connection.commit()
    
time_out = time.time()
print(time_out - time_in)

100%|██████████| 3555/3555 [00:09<00:00, 374.57it/s]

9.652346849441528


In [9]:
time_in = time.time()

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    for publicacao in tqdm(publicacoes):
        try:
            publicacao_titulo = publicacao['titulo']
            cursor.execute(f'SELECT publicacao_titulo_id FROM publicacao_titulo WHERE descricao LIKE "{publicacao_titulo}"')
            publicacao_titulo_id = cursor.fetchone()[0]
            cursor.fetchall()
            
            identificador = publicacao['identificador']
            ano = publicacao['ano']
            mes = publicacao['mes']
            conteudo = publicacao['publicacao']
            
#             cursor.execute(f'INSERT INTO publicacao(identificador, publicacao_titulo_id, ano, mes, conteudo)' +
#                            f'VALUES ({identificador}, {publicacao_titulo_id}, {ano}, {mes}, "{conteudo}")')
            cursor.execute('INSERT INTO publicacao(identificador, publicacao_titulo_id, ano, mes, conteudo) VALUES (%s, %s, %s, %s, %s)',
                          (identificador, publicacao_titulo_id, ano, mes, conteudo))
        except mysql.connector.Error as e:
            print('ERRO', e)
            cursor.close()
            connection.rollback()
            break
    cursor.close()
    connection.commit()
        
time_out = time.time()
print(time_out - time_in)

100%|██████████| 6593/6593 [00:11<00:00, 591.12it/s] 


11.527003049850464


In [10]:
time_in = time.time()

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    for licitacao in tqdm(licitacoes):
        if 'observacao' not in licitacao:
            continue
            
        try:
            identificador = licitacao['identificador']
            observacao = licitacao['observacao']
            
            cursor.execute('INSERT INTO observacao(identificador, conteudo) VALUES (%s, %s)',
                          (identificador, observacao))
        except mysql.connector.Error as e:
            print('ERRO', e)
            cursor.close()
            connection.rollback()
            break
    cursor.close()
    connection.commit()
        
time_out = time.time()
print(time_out - time_in)

100%|██████████| 3555/3555 [00:03<00:00, 1118.33it/s]


3.400820732116699


In [3]:
licitacoes = []
for licitacao in licitacoes_json:
    l = {}
    l['identificador'] = licitacao['identificador']
    l['titulo_modalidade'] = licitacao['titulo']['modalidade']
    l['titulo_numero'] = licitacao['titulo']['numero']
    l['titulo_ano'] = licitacao['titulo']['ano']
    l['tipo'] = licitacao['tipo']
    l['interessado'] = licitacao['interessado']
    l['processo_numero'] = licitacao['processo']['numero']
    l['processo_ano'] = licitacao['processo']['ano']
    if 'observacao' in licitacao:
        l['observacao'] = licitacao['observacao']
    licitacoes.append(l)

especificacoes = []
for licitacao in licitacoes_json:
    if 'especificacao' in licitacao:
        e = {}
        e['identificador'] = licitacao['identificador']
        e['especificacao'] = licitacao['especificacao']
        especificacoes.append(e)

publicacoes = []
for licitacao in licitacoes_json:
    if 'publicacoes' in licitacao:
        for publicacao in licitacao['publicacoes']:
            p = {}
            p['identificador'] = licitacao['identificador']
            p['titulo'] = publicacao['titulo']
            p['ano'] = publicacao['ano']
            p['mes'] = publicacao['mes']
            p['dia'] = publicacao['dia']
            p['publicacao'] = publicacao['conteudo']
            publicacoes.append(p)
            
documentos = []
for licitacao in licitacoes_json:
    if 'documentos' in licitacao:
        for documento in licitacao['documentos']:
            d = {}
            d['identificador'] = licitacao['identificador']
            d['nome'] = documento['nome']
            d['numero'] = documento['numero']
            if 'ano' in documento:
                d['ano'] = documento['ano']
            d['link'] = documento['link']
            
            documentos.append(d)

processos_apensados = []
for licitacao in licitacoes_json:
    if 'processos_apensados' in licitacao:
        for processo_apensado in licitacao['processos_apensados']:
            p = {}
            p['identificador'] = licitacao['identificador']
            p['numero'] = processo_apensado['numero']
            p['ano'] = processo_apensado['ano']
            processos_apensados.append(p)

prazos = []
for licitacao in licitacoes_json:
    if 'prazo_recebimento_propostas' in licitacao:
        prazo = licitacao['prazo_recebimento_propostas']
        p = {}
        
        p['identificador'] = licitacao['identificador']
        p['tipo'] = 'Recebimento Propostas'
        p['ano'] = prazo['ano']
        p['mes'] = prazo['mes']
        p['dia'] = prazo['dia']
        p['hora'] = prazo['hora']
        p['minuto'] = prazo['minuto']
        
        prazos.append(p)
        
    if 'prazo_apresentacao_propostas' in licitacao:
        prazo = licitacao['prazo_apresentacao_propostas']
        p = {}
        
        p['identificador'] = licitacao['identificador']
        p['tipo'] = 'Apresentação de Propostas'
        p['ano'] = prazo['ano']
        p['mes'] = prazo['mes']
        p['dia'] = prazo['dia']
        p['hora'] = prazo['hora']
        p['minuto'] = prazo['minuto']
        
        prazos.append(p)
        
    if 'prazo_entrega_envelopes' in licitacao:
        prazo = licitacao['prazo_entrega_envelopes']
        p = {}
        
        p['identificador'] = licitacao['identificador']
        p['tipo'] = 'Entrega dos Envelopes'
        p['ano'] = prazo['ano']
        p['mes'] = prazo['mes']
        p['dia'] = prazo['dia']
        p['hora'] = prazo['hora']
        p['minuto'] = prazo['minuto']
        
        prazos.append(p)
        
    if 'data_vencimento' in licitacao:
        prazo = licitacao['data_vencimento']
        p = {}
        
        p['identificador'] = licitacao['identificador']
        p['tipo'] = 'Data de vencimento'
        p['ano'] = prazo['ano']
        p['mes'] = prazo['mes']
        p['dia'] = prazo['dia']
        p['hora'] = prazo['hora']
        p['minuto'] = prazo['minuto']
        
        prazos.append(p)
        
    if 'data' in licitacao:
        prazo = licitacao['data']
        p = {}
        
        p['identificador'] = licitacao['identificador']
        p['tipo'] = 'Data'
        p['ano'] = prazo['ano']
        p['mes'] = prazo['mes']
        p['dia'] = prazo['dia']
        p['hora'] = prazo['hora']
        p['minuto'] = prazo['minuto']
        
        prazos.append(p)

In [3]:
df_licitacoes = pd.DataFrame(licitacoes)
df_especificacoes = pd.DataFrame(especificacoes)
df_publicacoes = pd.DataFrame(publicacoes)
df_documentos = pd.DataFrame(documentos)
df_apensados = pd.DataFrame(processos_apensados)
df_prazos = pd.DataFrame(prazos)

# print(df_licitacoes.info(), '\n')
# print(df_especificacoes.info(), '\n')
# print(df_publicacoes.info(), '\n')
# print(df_documentos.info(), '\n')
# print(df_apensados.info(), '\n')
# print(df_prazos.info(), '\n')

In [4]:
df_licitacoes[df_licitacoes['identificador'] == '4123']
df_publicacoes[df_publicacoes['identificador'] == '4123']

,identificador,titulo,ano,mes,dia,publicacao
1296,4123,Homologação,2017,04,27,NOTIFICAÇÃO DE ADJUDICAÇÃO E HOMOLOGAÇÃO - \r...


In [5]:
re_edital = re.compile('Edital', flags = re.IGNORECASE)
re_processo = re.compile('Processo', flags = re.IGNORECASE)
re_numero_ano = re.compile('((?P<numero>\d{1,3}(\.\d{3})?)/(?P<ano>\d{2,4}))')

lookahead = 20

#df_sample = df_publicacoes.sample(100)
    
def test(p, busca):
    pub = p['publicacao']
    busca_matches = busca.finditer(pub)
    busca_start = -1
    busca_end = 0
    
    for busca_match in busca_matches:
        new_str = pub[busca_match.end():busca_match.end() + lookahead]
        numero_matches = re_numero_ano.search(new_str)
        
        if numero_matches != None:
            busca_start = busca_match.start()
            busca_end = busca_match.end() + numero_matches.end()
            break
            
    if busca_end != 0:
        return True
    else:
        return False

df_tem_processo = df_publicacoes.apply(lambda p: test(p, re_processo), axis = 1)
df_tem_edital = df_publicacoes.apply(lambda p: test(p, re_edital), axis = 1)

In [6]:
df_tem = pd.DataFrame()
df_tem['identificador'] = df_publicacoes['identificador']
df_tem['tem_processo'] = df_tem_processo
df_tem['tem_edital'] = df_tem_edital
df_none = df_tem[(df_tem['tem_edital'] == False) & (df_tem['tem_processo'] == False)]